In [1]:
import nltk, re, pprint
from urllib import request
from collections import Counter
import numpy as np
import sklearn
url = 'http://www.gutenberg.org/cache/epub/1661/pg1661.txt'
response = request.urlopen(url)
raw = response.read().decode('utf8')

# Preprocessing

This section removes punctuation and miscellaneous text. It also adds the sentence start and ending tag. List for the short words obtained from: https://stackoverflow.com/questions/43018030/replace-appostrophe-short-words-in-python

In [2]:
replace = ['Project Gutenberg\'s The Adventures of Sherlock Holmes, by Arthur Conan Doyle',
       'This eBook is for the use of anyone anywhere at no cost and with',
       'almost no restrictions whatsoever.  You may copy it, give it away or',
       're-use it under the terms of the Project Gutenberg License included',
       'with this eBook or online at www.gutenberg.net',
       'Title: The Adventures of Sherlock Holmes',
       'Author: Arthur Conan Doyle',
       'Posting Date: April 18, 2011 [EBook #1661]',
       'First Posted: November 29, 2002', 'Language: English',
        '*** START OF THIS PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES ***',
        'Produced by an anonymous Project Gutenberg volunteer and Jose Menendez', 'THE ADVENTURES OF SHERLOCK HOLMES',
        'by', 'SIR ARTHUR CONAN DOYLE', 'I. A Scandal in Bohemia',
        'II. The Red-headed League', 'III. A Case of Identity', 'IV. The Boscombe Valley Mystery', 'V. The Five Orange Pips',
        'VI. The Man with the Twisted Lip', 'VII. The Adventure of the Blue Carbuncle', 'VIII. The Adventure of the Speckled Band',
        'IX. The Adventure of the Engineer\'s Thumb', 'X. The Adventure of the Noble Bachelor', 'XI. The Adventure of the Beryl Coronet',
        'XII. The Adventure of the Copper Beeches', ',',';',':','!','\"','\'','-']

for i in replace:
    raw.replace(i, '')
    
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he shall have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I would",
"i'd've": "I would have",
"i'll": "I will",
"i'll've": "I shall have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it shall have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she shall",
"she'll've": "she shall have",
"she's": "she has",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that has",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there has",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they shall",
"they'll've": "they shall have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall",
"what'll've": "what shall have",
"what're": "what are",
"what's": "what has",
"what've": "what have",
"when's": "when has",
"when've": "when have",
"where'd": "where did",
"where's": "where has",
"where've": "where have",
"who'll": "who shall",
"who'll've": "who shall have",
"who's": "who has",
"who've": "who have",
"why's": "why has",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you shall",
"you'll've": "you shall have",
"you're": "you are",
"you've": "you have"
}
for key in contractions.keys():
    raw.replace(key, contractions[key])
    
t = nltk.sent_tokenize(raw)

tt = []
size = 0
for line in t:
    temp = line.replace('\r', ' ')
    temp = temp.replace('\n', ' ')
    temp = '<s> ' + temp
    temp = temp.replace('.', ' </s>')
    temp = temp.replace('?', ' </s>')
    size += len(line.split())
    temp = temp.lower()
    tt.append(temp)

This part makes the train and test parts in the corpus

In [3]:
train, test = sklearn.model_selection.train_test_split(tt, train_size = 0.8, test_size = 0.2)

# Counting frequencies of n-grams
This part calculates the frequencies of unigram, bigram and trigrams present in the training set.

In [4]:
unigram_count = {}      #stores the unigram model
bigram_count = {}       #stores the bigram model
trigram_count = {}      #stores the trigram model
quadgram_count = {}     #stores the quadgram model

token_count = 0
word_count = {}         #single word count
two_word_count = {}     #double word count
three_word_count = {}   #three word count
four_word_count = {}    #four word count
two_word_count_generate = {}
total_word_count = 0


for line in train:
    for word in line.split():
        try:
            word_count[word] += 1
        except:
            word_count[word] = 1

            
types = list(word_count.keys())

for i in range(len(types)):
    for j in range(i+1, len(types)):
        two_word_count[(types[i], types[j])] = 0
        two_word_count[(types[j], types[i])] = 0
for i in word_count.keys():
    total_word_count += word_count[i]
    
    
for line in train:
    t = line.split()
    for i in range(1, len(t)):
        temp = (t[i-1], t[i])
        try:
            two_word_count_generate[temp] += 1
            two_word_count[temp] += 1
        except:
            two_word_count_generate[temp] = 1
            two_word_count[temp] = 1


for line in train:
    t = line.split()
    for i in range(2, len(t)):
        temp = (t[i-2], t[i-1], t[i])
        try:
            three_word_count[temp] += 1
        except:
            three_word_count[temp] = 1


for line in train:
    t = line.split()
    for i in range(3, len(t)):
        temp = (t[i-3], t[i-2], t[i-1], t[i])
        try:
            four_word_count[temp] += 1
        except:
            four_word_count[temp] = 1



for word in word_count.keys():
    token_count += word_count[word]


# Building MLE for n-gram models

In [5]:
def unigram():
    for line in train:
        temp = line.split()
        for i in range(0, len(temp)):
            temp1 = temp[i]
            if temp1 not in unigram_count.keys():
                unigram_count[temp1] = word_count[temp1]/total_word_count
def bigram():
    for line in train:
        temp = line.split()
        for i in range(1, len(temp)):
            temp1 = (temp[i-1], temp[i])
            if temp1 not in bigram_count.keys():
                bigram_count[temp1] = two_word_count[temp1]/word_count[temp[i-1]]
                
                
def trigram():
    for line in train:
        temp = line.split()
        for i in range(2, len(temp)):
            temp1 = (temp[i-2], temp[i-1], temp[i])
            if temp1 not in trigram_count.keys():
                trigram_count[temp1] = three_word_count[temp1]/two_word_count[(temp[i-2], temp[i-1])]

def quadgram():
    for line in train:
        temp = line.split()
        for i in range(3, len(temp)):
            temp1 = (temp[i-3], temp[i-2], temp[i-1], temp[i])
            if temp1 not in quadgram_count.keys():
                quadgram_count[temp1] = four_word_count[temp1]/three_word_count[(temp[i-3], temp[i-2], temp[i-1])]           


In [6]:
unigram()
bigram()
trigram()
quadgram()

# Sentence Generation using n-gram models

In [7]:
def generate(model):
    min_len = 8
    max_len = 15
    sent = []
    if model == "unigram":
        sent_length = 0
        while(sent_length < 15):
            temp = np.random.choice(list(unigram_count.keys()), 1, list(unigram_count.values()))[0]
            temp = str(temp)
            if temp == "</s>":
                if len(sent) >= 8:
                    sent.append(temp)
                    sent_length += 1
                    break
            else:
                sent_length += 1
                sent.append(temp)
        if sent_length == 15 and sent[14] != "</s>":
            sent.append("</s>")
    
    elif model == "bigram":
        sent.append("<s>")
        sent_length = 1
        attempts = 0
        while sent_length < 15:
            possible_words= []
            possible_probabilities = []
            for bigram in bigram_count.keys():
                if bigram[0] == sent[sent_length - 1]:
                    possible_words.append(bigram[1])
                    possible_probabilities.append(bigram_count[bigram])
            if len(possible_words) == 0:
                sentence = generate("bigram")
                return sentence
            temp = np.random.choice(possible_words, 1, possible_probabilities)[0]
            temp = str(temp)
            if temp == "</s>":
                attempts += 1
                if len(sent) >= 8:
                    sent.append(temp)
                    sent_length += 1
                    break
                elif attempts == 20:
                    sent.append(temp)
                    break
            else:
                attempts = 0
                sent_length += 1
                sent.append(temp)
        if sent_length == 15 and sent[14] != "</s>":
            sent.append("</s>")
            
    elif model == "trigram":
        possible_start = []
        possible_probabilities = []
        denominator = 0
        for val in two_word_count_generate.values():
            denominator += val
        for temp in two_word_count_generate.keys():
            if temp[0] == "<s>":
                possible_start.append(temp)
                possible_probabilities.append(two_word_count_generate[temp]/denominator)
        start = np.random.choice(list(range(len(possible_start))), 1, possible_probabilities)[0]
        sent.append(possible_start[start][0])
        sent.append(possible_start[start][1])
        sent_length = 2
        #print(sent)
        attempts = 0
        while sent_length < 15:
            #print(sent)
            possible_words= []
            possible_probabilities = []
            for trigram in trigram_count.keys():
                if trigram[0] == sent[sent_length - 2] and trigram[1] == sent[sent_length - 1]:
                    #print("HERE")
                    possible_words.append(trigram[2])
                    possible_probabilities.append(trigram_count[trigram])
            if len(possible_words) == 0:
                sentence = generate("trigram")
                return sentence
            temp = np.random.choice(possible_words, 1, possible_probabilities)[0]
            temp = str(temp)
            if temp == "</s>":
                attempts += 1
                if len(sent) >= 8:
                    sent.append(temp)
                    sent_length += 1
                    break
                elif attempts == 20:
                    sent.append(temp)
                    break
            else:
                attempts = 0
                sent_length += 1
                sent.append(temp)
        if sent_length == 15 and sent[14] != "</s>":
            sent.append("</s>")
        
    elif model == "quadgram":
        possible_start = []
        possible_probabilities = []
        denominator = 0
        for val in three_word_count.values():
            denominator += val
        for temp in three_word_count.keys():
            if temp[0] == "<s>":
                possible_start.append(temp)
                possible_probabilities.append(three_word_count[temp]/denominator)
        start = np.random.choice(list(range(len(possible_start))), 1, possible_probabilities)[0]
        sent.append(possible_start[start][0])
        sent.append(possible_start[start][1])
        sent.append(possible_start[start][2])
        sent_length = 3
        #print(sent)
        attempts = 0
        while sent_length < 15:
            #print(sent)
            possible_words= []
            possible_probabilities = []
            for quadgram in quadgram_count.keys():
                if quadgram[0] == sent[sent_length - 3] and quadgram[1] == sent[sent_length - 2] and quadgram[2] == sent[sent_length - 1]:
                    #print("HERE")
                    possible_words.append(quadgram[3])
                    possible_probabilities.append(quadgram_count[quadgram])
            if len(possible_words) == 0:
                sentence = generate("bigram")
                return sentence
            temp = np.random.choice(possible_words, 1, possible_probabilities)[0]
            temp = str(temp)
            if temp == "</s>":
                attempts += 1
                if len(sent) >= 8:
                    sent.append(temp)
                    sent_length += 1
                    break
                elif attempts == 20:
                    sent.append(temp)
                    break
            else:
                attempts = 0
                sent_length += 1
                sent.append(temp)
        if sent_length == 15 and sent[14] != "</s>":
            sent.append("</s>")
                    
        
    
    sentence = sent[0]
    for i in range(1, len(sent)):
        sentence = sentence + " " + sent[i]
    return sentence
        
        
        

These are the example sentences being generated:

## Unigram

In [8]:
for i in range(5):
    t = generate("unigram")
    t = t.replace('<s>', '')
    t = t.replace(' </s>', '.')
    print(t)

lap nursery deceased evolve upward condescend 'neither rose steps--for degenerating mission knee, white pays was--'and.
time holmes' "now transition save, village managed hansom, cheerful feasible "'ah, horror, assizes, calamity rending,.
puzzled drink returned, engine nonsense stamp smile clean-cut, arranged,' complimentary fowler act, deadly suburban bars.
"there waned "we've 'for knees, mice, china, lime-cream closed eavesdroppers metal, stepfather's burden curse proprietary.
children fare haste "oh, adventures staggered, gently arthur's arrived correspondent, pound "dear purveyor.h utilise.


## Bigram

In [9]:
for i in range(5):
    t = generate("bigram")
    t = t.replace('<s>', '')
    t = t.replace(' </s>', '.')
    print(t)

 anyhow, he answered; "i begin to lose.
 folk who will put so vile alley lurking behind.
 "in that colonel ushered in it is, however, which made inquiries are then roused.
 volunteers with damp and john swain, of uffa, and colleague, dr.
 some friend was late in at coburg square, and penetrating grey dressing-gown, a letter.


## Trigram

In [10]:
for i in range(5):
    t = generate("trigram")
    t = t.replace('<s>', '')
    t = t.replace(' </s>', '.')
    print(t)


 "we're hunting in couples again, doctor, you see," said i ruefully, pointing to a.
 "terse and to come with me, and it had cleared it all over, with.
 recently he has agreed to donate royalties under this one twelve months i find.
 "that sounds a little green-scummed pool, which is really managed by miss mary sutherland,.
 during all the terms of use and redistributing project gutenberg-tm trademark as set forth.


## Quadgram

In [11]:
for i in range(5):
    t = generate("quadgram")
    t = t.replace('<s>', '')
    t = t.replace(' </s>', '.')
    print(t)

 "well then, i shan't tell you.
 "'this is mr.
 here he comes.
 if horner were in danger it would be the blackest treachery to holmes to.
 he cried, throwing his fat hands out into the street.


# Probability

In [12]:
def probability(sentence, model):
    if model == "unigram":
        temp = sentence.split()
        ans = 0
        for word in temp:
            try:
                ans += np.log(unigram_count[word])
            except:
                return 0
    elif model == "bigram":
        temp = sentence.split()
        ans = 0
        for i in range(1, len(temp)):
            try:
                ans += np.log(bigram_count[(temp[i-1], temp[i])])
            except:
                return 0
    
    elif model == "trigram":
        temp = sentence.split()
        ans = 0
        for i in range(2, len(temp)):
            try:
                ans += np.log(trigram_count[(temp[i-2], temp[i-1], temp[i])])
            except:
                return 0
    elif model == "quadgram":
        temp = sentence.split()
        ans = 0
        for i in range(3, len(temp)):
            try:
                ans += np.log(quadgram_count[(temp[i-3], temp[i-2], temp[i-1], temp[i])])
            except:
                return 0
    return ans
            

This is an example of how the function works. The first input is the sentence and the second input is the model to be used to find the probability. The output is in log-space.

In [18]:
print(probability("i could see by his manner that he had stronger reasons for  satisfaction than his words alone would imply", "unigram"))
print(probability("i could see by his manner that he had stronger reasons for  satisfaction than his words alone would imply", "bigram"))
print(probability("i could see by his manner that he had stronger reasons for  satisfaction than his words alone would imply", "trigram"))
print(probability("i could see by his manner that he had stronger reasons for  satisfaction than his words alone would imply", "quadgram"))

-130.22575942365347
-62.75278951165759
-17.46298516544099
-5.886104031450156


# Add-one smoothing

In [14]:
new_bigram_model = {}
for i in range(len(types)):
    for j in range(i, len(types)):
        try:
            new_bigram_model[(types[i], types[j])] = (two_word_count[(types[i], types[j])] + 1)*word_count[types[i]]/(word_count[types[i]] + len(types))
        except:
            new_bigram_model[(types[i], types[j])] = (1)*word_count[types[i]]/(word_count[types[i]] + len(types))

for i in range(len(types)):
    for j in range(i, len(types)):
        try:
            new_bigram_model[(types[j], types[i])] = (two_word_count[(types[j], types[i])] + 1)*word_count[types[j]]/(word_count[types[j]] + len(types))
        except:
            new_bigram_model[(types[j], types[i])] = 1*word_count[types[j]]/(word_count[types[j]] + len(types))
            
        
            

In [18]:
del two_word_count_generate
highest_d = 0
second_d = 0
for key in two_word_count.keys():
    if abs(two_word_count[key] - new_bigram_model[key]) > highest_d:
        second_d = highest_d
        highest_d = abs(two_word_count[key] - new_bigram_model[key])

# Differences in count values
The largest differences in counts are listed below.

In [19]:
print(highest_d)
print(second_d)

55
47


# Good Turing Smoothing
The next few blocks implement the Good Turing Smoothing.

In [15]:
smooth_bigram_model = {}
freq_ofreq = {}
for freq in two_word_count.values():
    try:
        freq_ofreq[freq] += 1
    except:
        freq_ofreq[freq] = 1
        
max_bigram_count = len(types)*len(types)

new_freq_ofreq = {}
for freq in range(0, 11):
    new_freq_ofreq[freq] = (freq_ofreq[freq+1]*(freq+1))/freq_ofreq[freq]


# Perplexity values
The following block calculates the perplexity values. The perplexity was calculated using the method described here: https://stats.stackexchange.com/questions/129352/how-to-find-the-perplexity-of-a-corpus

In [16]:
temp = 0
for i in range(1, 11):
    temp += i - new_freq_ofreq[i]
D = temp/10
for freq in freq_ofreq.keys():
    if freq not in new_freq_ofreq.keys():
        new_freq_ofreq[freq] = freq - D

        
s_gt = 0
num_words = 0
s_ao = 0
for line in test:
    temp = line.split()
    num_words += len(temp)
    temp1 = 0
    temp5 = 0
    for i in range(1, len(temp)):
        try:
            temp3 = two_word_count[(temp[i-1], temp[i])]
            temp4 = new_bigram_model[(temp[i-1], temp[i])]
        except:
            temp3 = 0
            temp4 = 1/len(types)
        two_count = new_freq_ofreq[temp3]
        #print("GT ", two_count)
        #print("AO ", temp4)
        try:
            temp1 += np.log(two_count/word_count[temp[i-1]])
            temp5 += np.log(temp4/word_count[temp[i-1]])
        except:
            temp1 += np.log(two_count)
            temp5 += np.log(temp4)
    s_gt += temp1
    s_ao += temp5

print("Good Turing Perplexity: ", np.exp(-1*(s_gt/num_words)))
print("Add-one Perplexity: ", np.exp(-1*(s_ao/num_words)))
    

Good Turing Perplexity:  729.2924606834878
Add-one Perplexity:  2293.251455408744


In [17]:
train

['<s> at the second glance, however, i  perceived that there was a man standing in the southampton road,  a small bearded man in a grey suit, who seemed to be looking in  my direction </s>',
 '<s> i cried </s>',
 '<s> you must have started early, and yet you had  a good drive in a dog-cart, along heavy roads, before you reached  the station </s>"',
 '<s> there are only two, a  man and his wife </s>',
 '<s> i should not be very much surprised if this were  he whose step i hear now upon the stair </s>',
 '<s> "why did you come away to consult me in such a hurry </s>"',
 '<s> so there  were quarrels, and this, i am sure, was one of them </s>"',
 '<s> yet i  could not shake off the vague feeling of dread which it left  behind, though the sensation grew less keen as the weeks passed  and nothing happened to disturb the usual routine of our lives </s>',
 "<s> come!'",
 '<s> "no, i do not think so </s>',
 '<s> far away we could hear the  deep tones of the parish clock, which boomed out every 

In [ ]:
num_bi = 0
for i in two_word_count.values():
    if i != 0:
        num_bi += 1

num_tri = 0
for i in two_word_count.values():
    if i != 0:
        num_bi += 1
    
num_qu = 0
for i in two_word_count.values():
    if i != 0:
        num_bi += 1
    